In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Get player links

In [32]:
from IPython.display import clear_output
base_url = 'https://www.nbadraft.net/players/'

player_links = []

for page_num in range(1, 366):
    if page_num == 1:
        page_url = base_url
    else:
        page_url = f'{base_url}page/{page_num}/'
    
    # Send a GET request to the page URL
    response = requests.get(page_url)
    
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")
        
        # parent div containing all player links
        parent_div = soup.find('div', class_='wf-container')

        # Find all the player links on the page
        links = parent_div.find_all('a')
        
        # extract href attribute and append to player_links
        n = 0
        for link in links:
            link = link['href']
            if base_url in link:
                player_links.append(link)
                n += 1
        clear_output(wait=True)
        print(f'Found {n} player links in {page_url}')

Found 4 player links in https://www.nbadraft.net/players/page/365/


Save as a txt file because that took a while and we don't want to do it again

In [1]:
# with open('./data/nbadraft_player_links.txt', 'w') as file:
#     for link in player_links:
#         file.write(link + "\n")

Scrape player links for strengths/weaknesses

In [4]:
import numpy as np
f_links = './data/nbadraft_player_links.txt'

with open(f_links, 'r') as file:
    player_links = file.readlines()

player_links = [link.strip() for link in player_links]

In [8]:
# for some reason we have duplciates - only get unique
import numpy as np
player_links = list(np.unique(player_links))

In [38]:
from IPython.display import clear_output

data = []

for player_link in player_links:
    clear_output(wait=True)
    # player_link = 'https://www.nbadraft.net/players/amen-thompson/'
    player_name = player_link.rsplit('/',2)[-2]
    # player_name = player_name.replace('-', ' ')
    response = requests.get(player_link)

    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Find the paragraph elements starting with 'Strengths' and 'Weaknesses'
        try:
            strengths_paragraph = soup.find("strong", text="Strengths:").parent
            weaknesses_paragraph = soup.find("strong", text="Weaknesses:").parent
            year = soup.find('div',class_='col3').find('span', class_='label').text[:4]
        except AttributeError:
            continue
        
        # Extract the text content
        strengths_text = strengths_paragraph.text.replace("Strengths: ", "").strip()
        weaknesses_text = weaknesses_paragraph.text.replace("Weaknesses: ", "").strip()
        
        # Append the data to the list
        data.append({
            "player": player_name,
            'draft_year': year,
            "strengths": strengths_text,
            "weaknesses": weaknesses_text
        })
        print(f'Success! Scraped strengths/weaknesses for {player_name}')
    else:
        print(f"Failed to scrape data for {player_name}. Status code: {response.status_code}")
    # break

# Create a pandas dataframe from the scraped data
df = pd.DataFrame(data)

Sucess! Scraped strengths/weaknesses for ziga-samar


In [40]:
df.to_csv('./data/nbadraft_strengths_weaknesses.csv')

In [39]:
df

,player,draft_year,strengths,weaknesses
0,aaron-bradshaw,2024,7’0 center … Good size and length … Athletic w...,Has much room to get stronger in both upper an...
1,aaron-gordon,2014,A freakish athlete with incredible explosivene...,Gordon’s main weaknesses revolve around his la...
2,aaron-gray,Over,Unlike many post players Gray doesnt put the b...,Athletically Gray is not exceptionally quick o...
3,aaron-harrison,Big,Strengths: Leading scorer of a loaded 2015 Ken...,Weaknesses: Really struggled with consistency ...
4,aaron-henry,Big,Entering Tom Izzo’s storied program as a three...,"Despite his high scoring average, Henry was in..."
...,...,...,...,...
732,zhaire-smith,2018,Has an underrated feel for the game … High bas...,"As promising as he is, he’s still a project of..."
733,zhang-songtao,Over,Very coordinated big man with solid footwork …...,Release on his shot is rather slow … Jumps pre...
734,zhou-qi,2016,"Center prospect with great size, length and mo...",Lacks overall strength and weight to be effect...
735,ziaire-williams,2021,High level athlete who uses his size and verti...,Williams struggled to stay on the court for th...


In [23]:
player_links[0].rsplit('/',2)[-2]

'1253982'

In [25]:
df

,Player,Strengths,Weaknesses
0,amen-thompson,Thompson is a 6’7 210 perimeter player who is ...,Jump shot is a big concern currently (25 3FG%)...


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [14]:
links[0]['href']

'https://www.nbadraft.net/players/zvonimir-ivisic/'